# Kuala Lumpur Real Estate Price Prediction - Data Preparation

Use dataset published by Kaggle - https://www.kaggle.com/dragonduck/property-listings-in-kuala-lumpur - to train a simple deep learning model to predict prices for real estate in Kuala Lumpur.


This notebook contains the common data loading and preparation steps:
- load data from the input CSV
- do an assessment of the dataset to understand the number of distinct, missing, or invalid values by column
- clean up the columns of the dataset to deal with missing values, make numeric data available for training, and split columns with mixed types of data into distinct columns with consistent data types


# Common imports and variables
Imports and variable definitions that are common to the entire notebook


In [ ]:
!pip install requests
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime as dt
# common imports
import zipfile
import time
# import datetime, timedelta
import datetime
from datetime import datetime, timedelta
from datetime import date
from dateutil import relativedelta
from io import StringIO
import pandas as pd
import pickle
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from io import StringIO
import requests
import json
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import math
from subprocess import check_output
from IPython.display import display
import logging
import yaml
from collections import Counter
import re
import os
colab_path = "machine_learning_tabular_book/code/deep_learning_best_practices"


In [ ]:
!pwd

/content


In [ ]:
def get_config(config_file):
    ''' open config file with name config_file that contains parameters
    for this module and return Python object

    Args:
        config_file: filename containing config parameters

    Returns:
        config: Python dictionary with config parms from config file - dictionary


    '''
    current_path = os.getcwd()
    path_to_yaml = os.path.join(current_path, config_file)
    print("path_to_yaml " + path_to_yaml)
    try:
        with open(path_to_yaml, 'r') as c_file:
            config = yaml.safe_load(c_file)
        return config
    except Exception as error:
        print('Error reading the config file ' + str(error))

In [ ]:
def print_config_values(config):
    for val in config:
        print("config value ",val," ",str(config[val]))

In [ ]:
def set_current_directory():
  ''' check to see if the notebook is being run in Colab, and if so, set the current directory appropriately'''
  if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/notebooks


# Load Data
- ingest CSV into a Pandas dataframe 

In [ ]:
def get_path():
    ''' get the path for data files

    Returns:
        path: path for data directory

    '''
    rawpath = os.getcwd()
    # data is in a directory called "data" that is a sibling to the directory
    # containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
    return path

In [ ]:
def ingest_data(path,input_csv,pickled_input_dataframe,save_raw_dataframe,load_from_scratch):
    ''' load data into dataframe
    Args:
        path: path containing input file
        input_csv: input file name
        pickled_input_dataframe: pickled version of input file

    Returns:
        path: path for data directory
    '''
    if load_from_scratch:
        # if loading from scratch, the raw CSV file is expected to be in the data directory which is a sibling to the 
        # directory that contains this notebook
        unpickled_df = pd.read_csv(os.path.join(path,input_csv)) 
        if save_raw_dataframe:
            file_name = os.path.join(path,pickled_input_dataframe)
            print("file_name is ",file_name)
            unpickled_df.to_pickle(file_name)
    else:
        unpickled_df = pd.read_pickle(os.path.join(path,pickled_input_dataframe))
        logging.debug("reloader done")
    return(unpickled_df)

# Assess values
- assess columns for missing or invalid values

In [ ]:
def not_in_list(x, list):
    ''' check if a value is in a list
    Args:
        x: value to check
        list: list in which to check for the value

    Returns:
        retur_val: 1 if value is in not in list, 0 otherwise
    '''
    if x in list:
        return_val = 0
    else:
        return_val = 1
    return(return_val)
    

In [ ]:
def neg_val(x):
    ''' check if a value is in a list
    Args:
        x: value to check
    
    Returns:
        retur_val: 1 if value is negative, 0 otherwise
    '''
    if x >= 0:
        return_val = 0
    else:
        return_val = 1
    return(return_val)

In [ ]:
def basic_assessment(df,columns,valid_values,non_neg_continuous):
    ''' assess the values in a dataframe
    Args:
        df: dataframe for assessment
        columns: dictionary of column names by category
        valid_values: dictionary of valid values for categorical columns with limited number of valid values
        non_neg_continuous: list of continuous columns with only non-negative values as valid
    '''
    for col in list(df):
        print("Missing values in ",col," ",str(df[col].isna().sum()))
        print("Distinct values in ",col," ",str(df[col].nunique()))
    # for categorical columns with a limited number of valid values, count the number of invalid values by column
    for col in valid_values:
        print("non-valid values in column ",col," ",str(df[col].apply(lambda x:not_in_list(x,valid_values[col])).sum()))
    # count non-numeric values in continuous columns
    for col in columns['continuous']:
        # mask = pd.to_numeric(df['Hours_Worked'], errors='coerce').isna()
        mask = pd.to_numeric(df[col], errors='coerce').isna()
        print("non-numeric values in continuous col ",col," ",str(mask.sum()))
        # if there are no non-numeric values in the column and it muast have non-negative values, count negative values
        if (mask.sum()==0) and (col in non_neg_continuous):
            print("negative values in colum ",col," ",str(df[col].apply(lambda x:neg_val(x)).sum()))
    

In [ ]:
def out_of_range(x,max,min):
    ''' count whether a value is in a range
    Args:
        x: value to check in range
        max: top of the range to check
        min: bottom of the range to check
        
    Returns:
        ret_val: 1 if out of range, 0 otherwise
    '''
    if x > max or x < min:
        return_val = 1
    else:
        return_val = 0
    return(return_val)

In [ ]:
def remove_currency_symbol(currency_string, input_string):
    ''' remove the currency symbol from a string
    Args:
        currency_string: currency symbol
        input_string: string from which to remove currency symbol
        
    Returns:
        output_string: input_string with currency symbol removed
    '''    
    #print("input_string is",input_string)
    #print(" on length ",len(input_string))
    output_string = re.sub(currency_string,'',input_string)
    return(output_string)

In [ ]:
def remove_after_space(input_string):
  ''' remove everything after the first space in a string
  Args:
  input_string: string from which to remove content after space
  Returns:
  output_string: input_string with content removed
  '''    
  # remove leading and trailing spaces
  input_string = input_string.strip()
  # remove everything after internal spaces
  output_string = re.sub(r'\s* .*', '', input_string)
  output_string = re.sub(r'\([^)]*\)','',output_string)
  return(output_string)

In [ ]:
def clean_up_size_col(df,clean_up_list,size_bin_count):
    '''
    For the Size column in the KL real estate dataset:
    - lowercase values 
    - split the dimension values from the size type values
    - remove records with no numeric values
    - remove records with problematic values in clean_up_list
    Args:
      df: input dataframe
      clean_up_list: list of strings that, if a row contains them, the row should be removed from the dataset
    Returns:
      df: updated dataframe
    '''
    print("size_bin_count: ",size_bin_count)
    
    # remove rows with missing Size values
    df.dropna(subset=['Size'], inplace=True)

    # lowercase values in the Size column
    df['Size'] = df['Size'].str.lower()

    # split the Size column into two columns
    df[['Size_type','Size']] = df['Size'].str.split(':',expand=True)

    # replace missing values in the Size column with 0
    df['Size'] = df['Size'].fillna("0")
    
    # remove rows from Size that do not contain any digits
    df = df[df.Size.str.contains(r'\d')]
    # remove rows from Size that contain substrings from the clean_up_list
    for string in clean_up_list:
        df = df[~df.Size.str.contains(string,na=False)]
    # replace extraneous characters so that all Size entries are either numeric or of the form "numerica * numericb"
    df['Size'] = df['Size'].str.replace(',','').str.replace('`','').str.replace('@','x').str.replace('\+ sq. ft.','')
    df['Size'] = df['Size'].str.replace(' sq. ft.','').str.replace('sf sq. ft.','').str.replace('ft','').str.replace('sq','').str.replace("xx","*").str.replace("x ","*").str.replace(" x","*").str.replace("x","*").str.replace("X","*").replace('\'','')
    # remove extraneous characters following spaces
    df['Size'] = df['Size'].apply(lambda x: remove_after_space(x))
    # apply transformation to replace "numerica * numericb" with the result of the multiplication
    df['Size'] = df['Size'].apply(lambda x: eval(str(x)))
    df['Size'] = df['Size'].fillna(0.0)
    # define bin dimensions
    print("min is: ",df['Size'].min())
    print("max is: ",df['Size'].max())
    bins = np.linspace(df['Size'].min(), df['Size'].max(), size_bin_count)
    print("bins is: ",bins)
    # define bin names
    bin_labels = range(1,size_bin_count+1)
    print("bin_labels is: ",bin_labels)
    df['Size_bin'] = pd.qcut(df['Size'],size_bin_count, labels=bin_labels)
    df['Size_type_bin'] = df['Size_type']+df['Size_bin'].astype(str)
    return(df)

In [ ]:
def clean_up_price_col(df):
  '''
    For the Price column in the KL real estate dataset:
    - remove rows where the Price value is missing
    - remove currency symbol from remaining rows
    - convert values to float 
    Args:
      df: input dataframe
      clean_up_list: list of strings that, if a row contains them, the row should be removed from the dataset
    Returns:
      df: updated dataframe
  '''
  df.dropna(subset=['Price'], inplace=True)
  # remove currency symbol from remaining rows
  df['Price'] = df['Price'].apply(lambda x: remove_currency_symbol("RM ",x))


  # convert Price column to float
  df['Price'] = pd.to_numeric(df['Price'].str.replace(',',''), errors='coerce')
  return(df)

In [ ]:
def clean_up_rooms_col(df,treat_rooms_as_numeric):
  '''
    For the Rooms column in the KL real estate dataset:
    - if there is an operation in the value, conduct that operation
    - convert the value from string to integer
    Args:
      df: input dataframe
    Returns:
      df: updated dataframe
  '''
  if treat_rooms_as_numeric:
    print("Rooms treated as numeric")
    df['Rooms'] = df['Rooms'].fillna("0")
    # convert "6+" type values to "6+1"
    # lambda x: x if x%2==0 else x*2
    df['Rooms'] = df['Rooms'].apply(lambda x: x+"1" if x.endswith('+') else x)
    # "Studio", "20 Above" are the values in the Rooms column that cannot be dealt with as numeric
    df['Rooms'] = df['Rooms'].replace("Studio", "1")
    df['Rooms'] = df['Rooms'].replace("20 Above", "21")
    df['Rooms'] = df['Rooms'].apply(lambda x: eval(str(x)))
    df['Rooms'] = pd.to_numeric(df['Rooms'], errors='coerce')
    # replace missing values with 0
    df['Rooms'] = df['Rooms'].fillna(0)
  else:
    print("Rooms treated as non-numeric")
    df['Rooms'] = df['Rooms'].fillna("unknown_rooms")
  return(df)

In [ ]:
def clean_up_misc_cols(df,misc_col_dict):
  '''
    For columns where the only cleanup required is dealing with missing values, deal with missing values
    Args:
      df: input dataframe
    Returns:
      df: updated dataframe
  '''  
  for col in misc_col_dict:
    df[col] = df[col].fillna(misc_col_dict[col])
  return(df)

# Master cell
This cell contains calls to the other functions in this notebook to complete the data preparation

In [ ]:
# master cell to call the other functions
# get the path for data files
set_current_directory()
path = get_path()
print("path is ",path)
config = get_config('data_preparation_config.yml')
logging.getLogger().setLevel(logging.WARNING)
logging.warning("logging check")
#print_config_values(config)
# load dataframe and, if parameter set, save CSV file as a pickled dataframe
df = ingest_data(path,config['file_names']['input_csv'],config['file_names']['pickled_input_dataframe'],config['general']['save_raw_dataframe'],config['general']['load_from_scratch'])
print("shape ",df.shape)
'''
# get basic assessment information for the dataframe
basic_assessment(df,config['columns'],config['valid_values'],config['non_negative_continuous'])
# get assessment for geospatial information
geo_assessment(df,config['bounding_box'])
'''
df.head()

Mounted at /content/drive
/content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/notebooks
path is  /content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/data
path_to_yaml /content/drive/MyDrive/machine_learning_tabular_book/code/deep_learning_best_practices/notebooks/data_preparation_config.yml


shape  (53883, 8)


,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,"KLCC, Kuala Lumpur","RM 1,250,000",2+1,3.0,2.0,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,"Damansara Heights, Kuala Lumpur","RM 6,800,000",6,7.0,NaN,Bungalow,Land area : 6900 sq. ft.,Partly Furnished
2,"Dutamas, Kuala Lumpur","RM 1,030,000",3,4.0,2.0,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
3,"Cheras, Kuala Lumpur",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bukit Jalil, Kuala Lumpur","RM 900,000",4+1,3.0,2.0,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished


In [ ]:
# get number of unique values in each column
counts = df.nunique()
print("unique value counts:\n",counts)

unique value counts:
 Location          112
Price            4280
Rooms              43
Bathrooms          17
Car Parks          21
Property Type      99
Size             6190
Furnishing          4
dtype: int64


In [ ]:
df['Rooms'].value_counts()

3           14249
3+1          8070
2            5407
4            5018
4+1          4404
5+1          2340
1            2322
5            2065
2+1          1938
1+1          1191
6             937
Studio        874
6+1           807
4+2           479
3+2           477
5+2           410
7             358
7+1           237
2+2           132
8             125
6+             86
8+1            64
9              40
7+             32
10             25
9+1            14
8+             14
10+             8
12              8
20 Above        8
1+2             8
11              6
10+1            4
12+             4
13+             3
11+1            3
14              2
9+              2
16              2
15+             1
18              1
13+1            1
13              1
Name: Rooms, dtype: int64

In [ ]:
df['Property Type'].value_counts()

Condominium                               11552
Serviced Residence                         7207
Condominium (Corner)                       5736
Condominium (Intermediate)                 5134
Serviced Residence (Intermediate)          3269
                                          ...  
3-sty Terrace/Link House (Triplex)            1
2-sty Terrace/Link House (Penthouse)          1
Bungalow (Triplex)                            1
2.5-sty Terrace/Link House (Penthouse)        1
4.5-sty Terrace/Link House (Corner)           1
Name: Property Type, Length: 99, dtype: int64

In [ ]:
df['Size'].value_counts()

Land area : 22x75 sq. ft.    615
Built-up : 1,100 sq. ft.     543
Built-up : 950 sq. ft.       497
Built-up : 850 sq. ft.       383
Built-up : 1,200 sq. ft.     381
                            ... 
Land area : 7587 sq. ft.       1
Land area : 4146 sq. ft.       1
Land area : 1167 sq. ft.       1
Land area : 7287 sq. ft.       1
Land area : 1544 sq. ft.       1
Name: Size, Length: 6190, dtype: int64

In [ ]:
count = (df['Size'].str.contains('@')).sum()

print("count of Size values containing @: ",count)

count of Size values containing @:  1


In [ ]:
count = (df['Size'].str.contains('x')).sum() + (df['Size'].str.contains('X')).sum()

print("count of Size values containing x is: ",count)

count of Size values containing x is:  5564


In [ ]:
# show examples of rows where the area part of Size is expressed as length x width
df_length_width = df[df['Size'].str.contains('x') | df['Size'].str.contains('X')]
df_length_width.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
8,"Taman Tun Dr Ismail, Kuala Lumpur","RM 1,950,000",4+1,4.0,3.0,2-sty Terrace/Link House (EndLot),Land area : 25x75 sq. ft.,Partly Furnished
11,"Taman Tun Dr Ismail, Kuala Lumpur","RM 1,680,000",4,3.0,NaN,2-sty Terrace/Link House (Intermediate),Land area : 22 x 80 sq. ft.,Partly Furnished
25,"Sungai Besi, Kuala Lumpur",NaN,5,4.0,NaN,3-sty Terrace/Link House (Intermediate),Land area : 20x75 sq. ft.,Fully Furnished
32,"Kepong, Kuala Lumpur","RM 560,000",3,2.0,NaN,2-sty Terrace/Link House (Intermediate),Land area : 16x55 sq. ft.,Partly Furnished
45,"Sungai Besi, Kuala Lumpur","RM 900,000",6,5.0,3.0,3-sty Terrace/Link House (Intermediate),Land area : 20 X 80 sq. ft.,Fully Furnished


In [ ]:
df_problematic = df[df['Size'].str.contains('\+') | df['Size'].str.contains('@')]
df_problematic.head()

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
362,"Sungai Besi, Kuala Lumpur","RM 1,600,000",4+1,6.0,4.0,3-sty Terrace/Link House (EndLot),Land area : 10+24 x 80 sq. ft.,Partly Furnished
450,"Sungai Besi, Kuala Lumpur","RM 2,250,000",6,5.0,7.0,3-sty Terrace/Link House (Corner),Land area : 40+30 X 80 sq. ft.,Partly Furnished
817,"Sungai Besi, Kuala Lumpur","RM 2,000,000",4+1,6.0,6.0,Semi-detached House (Corner),Land area : 20+26 x 80 sq. ft.,Partly Furnished
6926,"Kepong, Kuala Lumpur","RM 950,000",4,3.0,NaN,2-sty Terrace/Link House (Intermediate),Land area : 22+5x80 sq. ft.,Partly Furnished
8841,"Bukit Jalil, Kuala Lumpur","RM 2,500,000",4+1,4.0,2.0,2.5-sty Terrace/Link House (Corner),Land area : 25X85+17X85 sq. ft.,Partly Furnished


In [ ]:
missing_values_count = df.isnull().sum()
print("missing values before cleanup:\n",missing_values_count)

missing values before cleanup:
 Location             0
Price              248
Rooms             1706
Bathrooms         2013
Car Parks        17567
Property Type       25
Size              1063
Furnishing        6930
dtype: int64


In [ ]:
# for rows where the key columns are all missing values, drop the row
# continuous columns:  Index(['Price', 'Bathrooms', 'Car Parks', 'Size'], dtype='object')
# categorical columns:  Index(['Location', 'Rooms', 'Property Type', 'Furnishing', 'Size_type'], dtype='object')
# cleanup overview:
# Rooms: clean_up_rooms_col()
# Size / Size_type: clean_up_size_col()
# Price: clean_up_price_col()
# Bathrooms, Car Parks, Furnishing, Property Type, Location: clean_up_misc_cols()
# for columns that are mandatory for making a price prediction, drop rows that are missing values in these columns
key_cols = ['Rooms','Bathrooms','Furnishing']
df = df.dropna(subset=key_cols, how='all')


In [ ]:
df.describe()

,Bathrooms,Car Parks
count,51870.000000,36285.000000
mean,3.073434,2.006973
std,1.631079,1.306358
min,1.000000,1.000000
25%,2.000000,1.000000
50%,2.000000,2.000000
75%,4.000000,2.000000
max,20.000000,30.000000


In [ ]:
# complete clean up of Price and Size columns
df = clean_up_price_col(df)
clean_up_list = ["-","\+",'\'','\~',"xx","sf","acre","#"]
df = clean_up_size_col(df,clean_up_list,config['general']['size_bin_count'])
print("shape ",df.shape)
df.head()

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


size_bin_count:  10


In [ ]:
rows = df.loc[df['Rooms'] == 'Studio']


In [ ]:
rows

In [ ]:
# clean up the Rooms column
df = clean_up_rooms_col(df, config['general']['treat_rooms_as_numeric'])
df.head()

In [ ]:
# define dictionary of columns that will get basic replacements
misc_col_dict = config['misc_col_dict']
# clean up the columns where we just want to deal with missing values
df = clean_up_misc_cols(df,misc_col_dict)

In [ ]:
path

In [ ]:
# if specified, save the transformed dataset as a pickle file
if config['general']['save_transformed_dataframe']:
    print("path is ",path)
    file_name = os.path.join(path,config['file_names']['pickled_output_dataframe'])
    print("file_name is ",file_name)
    df.to_pickle(file_name)

In [ ]:
df.columns

In [ ]:
# dataset columns by type
# continuous
# categorical
categorical_columns = df.select_dtypes(include=['object']).columns
continuous_columns = continuous_columns = df.select_dtypes(include=['float', 'int']).columns
print("continuous columns: ",continuous_columns)
print("categorical columns: ",categorical_columns)

In [ ]:
df['Location'].value_counts()

In [ ]:
# confirm that all missing values have been dealt with
missing_values_count = df.isnull().sum()
print("missing values post cleanup: ",missing_values_count)

In [ ]:
df.describe()

In [ ]:
df["Price"].median()

In [ ]:
df["Price"].mean()

In [ ]:
# check the distribution of unique values in each column post cleanup
# continuous columns:  Index(['Price', 'Bathrooms', 'Car Parks', 'Size'], dtype='object')
# categorical columns:  Index(['Location', 'Rooms', 'Property Type', 'Furnishing', 'Size_type'], dtype='object')
counts = df.nunique()
print("unique value counts: ",counts)
print("shape ",df.shape)


In [ ]:
df['Size_type_bin'].value_counts()

In [ ]:
df['Size_bin'].value_counts()